In [2]:
import mlflow 
try:
  experiment_id = mlflow.create_experiment('/Users/srijith.rajamohan/Documents/Development/MLflow/mlflow_training/')
  experiment = mlflow.get_experiment(experiment_id)
except:
  experiment_id = 11791967
  
mlflow.autolog(log_input_examples=True, log_models=True, exclusive=False)

In [3]:
import pandas as pd
from sklearn.model_selection import *
input_df = pd.read_csv("/Users/srijith.rajamohan/Documents/Development/MLflow/mlflow_training/WA_Fn-UseC_-Telco-Customer-Churn.csv")
#input_df = spark.table("ibm_telco_churn.bronze_customers")
train_df, test_df = train_test_split(input_df, test_size=0.1, random_state=42)
#use later for additional inference testing
new_df = test_df 
display(train_df)

2021/11/01 21:29:04 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/srijith.rajamohan/opt/anaconda3/envs/mlflow_demos/lib/python3.8/site-packages/setuptools/distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first."
2021/11/01 21:29:13 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
1554,9313-QOLTZ,Male,0,No,No,26,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Month-to-month,No,Mailed check,20.90,518.3,No
4817,6654-QGBZZ,Female,0,No,No,1,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Month-to-month,No,Mailed check,19.90,19.9,No
5957,6877-LGWXO,Male,1,Yes,No,18,Yes,No,Fiber optic,No,...,No,No,No,Yes,Month-to-month,No,Electronic check,78.55,1422.65,Yes
5123,8020-BWHYL,Female,1,No,No,15,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Credit card (automatic),75.30,1147.45,Yes
3217,3594-IVHJZ,Female,0,No,No,2,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Month-to-month,Yes,Mailed check,20.45,42.45,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3772,0684-AOSIH,Male,0,Yes,No,1,Yes,No,Fiber optic,Yes,...,No,No,Yes,Yes,Month-to-month,Yes,Electronic check,95.00,95,Yes
5191,5982-PSMKW,Female,0,Yes,Yes,23,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),91.10,2198.3,No
5226,8044-BGWPI,Male,0,Yes,Yes,12,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Month-to-month,Yes,Electronic check,21.15,306.05,No
5390,7450-NWRTR,Male,1,No,No,12,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.45,1200.15,Yes


In [4]:
#test_df = test_df.withColumn("churn", when(test_df.Churn == 'Yes' ,1).otherwise(0))
test_df['churn'] = test_df['Churn'].apply(lambda x: 1 if x == 'Yes' else 0)
#train_df = train_df.withColumn("churn", when(train_df.Churn == 'Yes' ,1).otherwise(0))
train_df['churn'] = train_df['Churn'].apply(lambda x: 1 if x == 'Yes' else 0)

<ipython-input-4-ddc604d0e23b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['churn'] = test_df['Churn'].apply(lambda x: 1 if x == 'Yes' else 0)
<ipython-input-4-ddc604d0e23b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['churn'] = train_df['Churn'].apply(lambda x: 1 if x == 'Yes' else 0)


In [5]:
train_df.drop(labels='Churn', axis=1)
test_df.drop(labels='Churn', axis=1)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,churn
185,1024-GUALD,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,24.80,24.8,1
2715,0484-JPBRU,Male,0,No,No,41,Yes,Yes,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Month-to-month,Yes,Bank transfer (automatic),25.25,996.45,0
3825,3620-EHIMZ,Female,0,Yes,Yes,52,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,19.35,1031.7,0
1807,6910-HADCM,Female,0,No,No,1,Yes,No,Fiber optic,No,...,Yes,No,No,No,Month-to-month,No,Electronic check,76.35,76.35,1
132,8587-XYZSF,Male,0,No,No,67,Yes,No,DSL,No,...,No,Yes,No,No,Two year,No,Bank transfer (automatic),50.55,3260.1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586,4568-KNYWR,Male,0,No,No,52,Yes,No,DSL,Yes,...,No,No,Yes,No,Two year,Yes,Credit card (automatic),63.25,3342.45,0
4503,9658-WYUFB,Female,0,No,No,17,Yes,Yes,Fiber optic,Yes,...,No,No,Yes,No,Month-to-month,Yes,Electronic check,94.40,1617.5,1
5283,0997-YTLNY,Female,0,No,Yes,19,Yes,No,DSL,No,...,No,Yes,No,No,Month-to-month,No,Mailed check,48.80,953.65,0
3608,3208-YPIOE,Male,0,No,No,39,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,No,Electronic check,75.25,3017.65,1


In [6]:
import sklearn.metrics
import numpy as np
y_test = test_df["churn"]
X_test = test_df.drop("churn", axis=1)

In [7]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

transformers = []
numerical_pipeline = Pipeline(steps=[
    ("converter", FunctionTransformer(lambda df: df.apply(pd.to_numeric, errors="coerce"))),
    ("imputer", SimpleImputer(strategy="mean"))
])

transformers.append(("numerical", numerical_pipeline, ["MonthlyCharges", "TotalCharges"]))

In [8]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

one_hot_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(missing_values=None, strategy="constant", fill_value="")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

transformers.append(("onehot", one_hot_pipeline, ["Contract", "Dependents", "DeviceProtection", "InternetService", "MultipleLines", "OnlineBackup", "OnlineSecurity", "PaperlessBilling", "Partner", "PaymentMethod", "PhoneService", "SeniorCitizen", "StreamingMovies", "StreamingTV", "TechSupport", "gender", "tenure"]))

In [9]:
from sklearn.feature_extraction import FeatureHasher
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

for feature in ["customerID"]:
    hash_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(missing_values=None, strategy="constant", fill_value="")),
        (f"{feature}_hasher", FeatureHasher(n_features=1024, input_type="string"))])
    transformers.append((f"{feature}_hasher", hash_transformer, [feature]))

In [10]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(transformers, remainder="passthrough", sparse_threshold=0)

In [11]:
from sklearn.preprocessing import StandardScaler

standardizer = StandardScaler()

In [12]:
from sklearn.model_selection import train_test_split

df_loaded = train_df
target_col = "churn"
split_X = df_loaded.drop([target_col], axis=1)
split_y = df_loaded[target_col]

X_train, X_val, y_train, y_val = train_test_split(split_X, split_y, random_state=398741429, stratify=split_y)

In [21]:
import mlflow
import sklearn
from xgboost import XGBClassifier
from sklearn import set_config
from sklearn.pipeline import Pipeline

#set_config(display="diagram")

xgbc_classifier = XGBClassifier(
  learning_rate=0.05064395325585373,
  max_depth=3,
  min_child_weight=8,
  subsample=0.9602619596041891,
  random_state=398741429,
)

model = Pipeline([
    ("preprocessor", preprocessor),
   # ("standardizer", standardizer),
   # ("classifier", xgbc_classifier),
])

model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough', sparse_threshold=0,
                                   transformers=[('numerical',
                                                  Pipeline(steps=[('converter',
                                                                   FunctionTransformer(func=<function <lambda> at 0x7ff3dd9bbb80>)),
                                                                  ('imputer',
                                                                   SimpleImputer())]),
                                                  ['MonthlyCharges',
                                                   'TotalCharges']),
                                                 ('onehot',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='',
                                                                          

In [24]:
import pandas as pd
with mlflow.start_run(run_name="xgboost") as mlflow_run:
    model.fit(X_train, y_train)
    
    # Training metrics are logged by MLflow autologging
    # Log metrics for the validation set
    #xgbc_val_metrics = mlflow.sklearn.eval_and_log_metrics(model, X_val, y_val,
    #                                                            prefix="val_")
    #display(pd.DataFrame(xgbc_val_metrics, index=[0]))

2021/11/01 22:20:22 WARNING mlflow.utils: Truncated the value of the key `steps`. Truncated value: `[('preprocessor', ColumnTransformer(remainder='passthrough', sparse_threshold=0,
                  transformers=[('numerical',
                                 Pipeline(steps=[('converter',
                                                  Functio...`
2021/11/01 22:20:22 WARNING mlflow.utils: Truncated the value of the key `preprocessor`. Truncated value: `ColumnTransformer(remainder='passthrough', sparse_threshold=0,
                  transformers=[('numerical',
                                 Pipeline(steps=[('converter',
                                                  FunctionTransformer(func=...`
2021/11/01 22:20:22 WARNING mlflow.utils: Truncated the value of the key `preprocessor__transformers`. Truncated value: `[('numerical', Pipeline(steps=[('converter',
                 FunctionTransformer(func=<function <lambda> at 0x7ff3dd9bbb80>)),
                ('imputer', SimpleImpute

In [ ]:
mlflow.sklearn.save_model(model, "my_model_sr")

In [19]:
y_train.value_counts()

0    3497
1    1256
Name: churn, dtype: int64

In [20]:
X_train

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
5662,3244-DCJWY,Female,0,No,No,1,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.25,70.25,Yes
508,4636-QRJKY,Female,0,Yes,Yes,6,Yes,Yes,Fiber optic,No,...,No,Yes,No,No,Month-to-month,Yes,Electronic check,80.50,502.85,Yes
1758,5848-FHRFC,Female,0,No,No,68,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Bank transfer (automatic),20.05,1386.9,No
122,0404-SWRVG,Male,0,No,No,3,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,74.40,229.55,Yes
3041,1406-PUQVY,Male,0,No,Yes,1,Yes,No,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,49.90,49.9,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4080,6199-IPCAO,Female,0,Yes,Yes,29,Yes,Yes,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,26.10,692.55,No
1059,0376-YMCJC,Male,0,No,No,23,Yes,Yes,Fiber optic,No,...,No,Yes,Yes,No,Month-to-month,Yes,Electronic check,90.60,1943.2,Yes
1236,5835-BEQEU,Male,0,No,No,3,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,76.95,228.4,Yes
5984,9185-TQCVP,Male,0,Yes,No,14,Yes,Yes,Fiber optic,No,...,No,No,No,Yes,Month-to-month,Yes,Electronic check,85.15,1139.2,Yes
